**Интерфейс для удобного общения с виртуальным помощником:**

In [6]:
import gradio as gr
import clickhouse_connect
from embedding import E5LargeEmbeddingFunction
from similarity import bm25_ensemble_with_crossenc_answer, prep_query, bm25_fewshot_with_cross_encoder
from pprint import pprint
import requests
import json
import pandas as pd
import gc


client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')
emb_func = E5LargeEmbeddingFunction()

def message_llm(system_prompt, user_input, temperature=0.6):
    '''Function to message (test) our LLM'''

    try:
        url = "https://live-relaxed-oryx.ngrok-free.app/v1/chat/completions"

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            { "role": "user", "content": user_input}
          ],
          "temperature": 0.6,
          # "max_tokens": -1,
          # "stream": False
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        
        print(f"Error {e}!")
        
rag_prompt = """
### CONTEXT ###

1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### USER ###

На основании источников 1,2,3 дай исчерпывающий ответ на
приведенный ниже вопрос. Если ты неуверен, что можешь дать ответ на основании 
источников 1,2,3 пиши «Я не могу ответить на этот вопрос.».
- {QUESTION}
"""

few_shot_data = pd.read_csv('data/finetune_df_final.csv', index_col=0)
few_shot_data

rag_prompt_fw_ex = """
### EXAMPLES ###

1) Пример 1:
КОНТЕКСТ: {context_1}
ВОПРОС: {question_1}
ОТВЕТ: {answer_1}

2) Пример 2:
КОНТЕКСТ: {context_2}
ВОПРОС: {question_2}
ОТВЕТ: {answer_2}

3) Пример 3:
КОНТЕКСТ: {context_3}
ВОПРОС: {question_3}
ОТВЕТ: {answer_3}

### CONTEXT ###

1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### USER ###

На основании источников 1,2,3 дай исчерпывающий ответ на
приведенный ниже вопрос. Если ты неуверен, что можешь дать ответ на основании 
источников 1,2,3 пиши «Я не могу ответить на этот вопрос.».
- {QUESTION}
"""

sc_prompt = """
### CONTEXT ###

1) МНЕНИЕ 1: 
- {answer1}

2) МНЕНИЕ 2: 
- {answer2}

3) МНЕНИЕ 3:
- {answer3}

### USER ###
На основании мнений 1,2,3 дай итоговый ответ по принципу большинства.
"""

system_prompt = """
### SYSTEM ###
Ты - дружелюбный ассистент Центрального банка Российской Федерации. 
Исчерпывающе отвечай клиентам на возникшие вопросы.
На все вопросы не касающиеся ЦБ РФ, финансов и экономики ты обязан отвечать:
'Давайте сменим тему.'
"""

def ask_chatbot(mode, question):
    gc.collect()
    try:
        formatted_rag_prompt = ""
        query = prep_query(question)
        if mode == "zero-shot":

            result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

            type_of_source1, type_of_source2, type_of_source3  = '', '', ''
            link_to_source1, link_to_source2, link_to_source3 = '', '', ''
            batch_1, batch_2, batch_3 = '', '', ''


            for idx, batch in enumerate(start=1, iterable=result):
                if idx==1:
                    if batch[5]:
                        type_of_source1 = 'ТАБЛИЦА'
                    else:
                        type_of_source1 = 'ТЕКСТ'
                    link_to_source1 = batch[2]
                    batch_1 = batch[3]

                if idx==2:
                    if batch[5]:
                        type_of_source2 = 'ТАБЛИЦА'
                    else:
                        type_of_source2 = 'ТЕКСТ'
                    link_to_source2 = batch[2]
                    batch_2 = batch[3]

                if idx==3:
                    if batch[5]:
                        type_of_source3 = 'ТАБЛИЦА'
                    else:
                        type_of_source3 = 'ТЕКСТ'
                    link_to_source3 = batch[2]
                    batch_3 = batch[3]


            formatted_rag_prompt = rag_prompt.format(
                type_of_source1=type_of_source1,
                link_to_source1=link_to_source1,
                batch_1=batch_1,
                type_of_source2=type_of_source2,
                link_to_source2=link_to_source2,
                batch_2=batch_2,
                type_of_source3=type_of_source3,
                link_to_source3=link_to_source3,
                batch_3=batch_3,
                QUESTION=question
            )

            answer = message_llm(system_prompt, formatted_rag_prompt)

            if not ('Давайте сменим тему.' in answer or 'Я не могу ответить на этот вопрос' in answer):
                return f"{answer}\n\nИсточники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}"
            else:
                return answer
            
        elif mode == "few-shot":
            result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

            type_of_source1, type_of_source2, type_of_source3  = '', '', ''
            link_to_source1, link_to_source2, link_to_source3 = '', '', ''
            batch_1, batch_2, batch_3 = '', '', ''

            for idx, batch in enumerate(start=1, iterable=result):
                if idx==1:
                    if batch[5]:
                        type_of_source1 = 'ТАБЛИЦА'
                    else:
                        type_of_source1 = 'ТЕКСТ'
                    link_to_source1 = batch[2]
                    batch_1 = batch[3]

                if idx==2:
                    if batch[5]:
                        type_of_source2 = 'ТАБЛИЦА'
                    else:
                        type_of_source2 = 'ТЕКСТ'
                    link_to_source2 = batch[2]
                    batch_2 = batch[3]

                if idx==3:
                    if batch[5]:
                        type_of_source3 = 'ТАБЛИЦА'
                    else:
                        type_of_source3 = 'ТЕКСТ'
                    link_to_source3 = batch[2]
                    batch_3 = batch[3]

            few_shot_examples = bm25_fewshot_with_cross_encoder(bm25_n_results=100, cr_enc_n_results=3, df=few_shot_data, query=query)

            formatted_rag_prompt = rag_prompt_fw_ex.format(
                context_1=few_shot_examples[0]['context'],
                question_1=few_shot_examples[0]['question'],
                answer_1=few_shot_examples[0]['answer'],
                context_2=few_shot_examples[1]['context'],
                question_2=few_shot_examples[1]['question'], 
                answer_2=few_shot_examples[1]['answer'],
                context_3=few_shot_examples[2]['context'],
                question_3=few_shot_examples[2]['question'],  
                answer_3=few_shot_examples[2]['answer'],  
                type_of_source1=type_of_source1,
                link_to_source1=link_to_source1,
                batch_1=batch_1,
                type_of_source2=type_of_source2,
                link_to_source2=link_to_source2,
                batch_2=batch_2,
                type_of_source3=type_of_source3,
                link_to_source3=link_to_source3,
                batch_3=batch_3,
                QUESTION=question
            )
            answer = message_llm(system_prompt, formatted_rag_prompt)

            if not ('Давайте сменим тему.' in answer or 'Я не могу ответить на этот вопрос' in answer):
                return f"{answer}\n\nИсточники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}"
            else:
                return answer
            
        elif mode == "self-consistency":
            result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

            type_of_source1, type_of_source2, type_of_source3  = '', '', ''
            link_to_source1, link_to_source2, link_to_source3 = '', '', ''
            batch_1, batch_2, batch_3 = '', '', ''


            for idx, batch in enumerate(start=1, iterable=result):
                if idx==1:
                    if batch[5]:
                        type_of_source1 = 'ТАБЛИЦА'
                    else:
                        type_of_source1 = 'ТЕКСТ'
                    link_to_source1 = batch[2]
                    batch_1 = batch[3]

                if idx==2:
                    if batch[5]:
                        type_of_source2 = 'ТАБЛИЦА'
                    else:
                        type_of_source2 = 'ТЕКСТ'
                    link_to_source2 = batch[2]
                    batch_2 = batch[3]

                if idx==3:
                    if batch[5]:
                        type_of_source3 = 'ТАБЛИЦА'
                    else:
                        type_of_source3 = 'ТЕКСТ'
                    link_to_source3 = batch[2]
                    batch_3 = batch[3]

            formatted_rag_prompt = rag_prompt.format(
                type_of_source1=type_of_source1,
                link_to_source1=link_to_source1,
                batch_1=batch_1,
                type_of_source2=type_of_source2,
                link_to_source2=link_to_source2,
                batch_2=batch_2,
                type_of_source3=type_of_source3,
                link_to_source3=link_to_source3,
                batch_3=batch_3,
                QUESTION=question
            )

            answer1 = message_llm(system_prompt, formatted_rag_prompt)
            answer2 = message_llm(system_prompt, formatted_rag_prompt)
            answer3 = message_llm(system_prompt, formatted_rag_prompt)

            formatted_som_prompt = sc_prompt.format(answer1=answer1,answer2=answer2,answer3=answer3)
            final_answer = message_llm(system_prompt, formatted_som_prompt)
            
            if not ('Давайте сменим тему.' in final_answer or 'Я не могу ответить на этот вопрос' in final_answer):
                return f"{final_answer}\n\nИсточники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}"
            else:
                return final_answer
    except Exception as e:
        return f"Пожалуйста, посетите FAQ страницу сайта ЦБ РФ (https://www.cbr.ru/faq/) для получения ответа на этот вопрос."  


iface = gr.Interface(
    fn=ask_chatbot,
    inputs=[
        gr.Dropdown(choices=["zero-shot", "few-shot", "self-consistency"], label="Выберите режим"),
        gr.Textbox(lines=2, placeholder="Введите ваш вопрос здесь...", label="Вопрос")
    ],
    outputs=gr.Textbox(label="Ответ"),
    title="Чат с виртуальным помощником - SCOOF DEVELOPERS",
    description="Выберите режим и введите ваше сообщение, чтобы получить ответ от модели:"
)

iface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [2]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 19.7 MB/s eta 0:00:0000:0100:01
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 KB 19.3 MB/s eta 0:00:00
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached altair-5.2.0-py3-none-any.whl (996 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 13.3 MB/s eta 0:00:0000:0100:01
  Using cached websockets-11.0.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (129 kB)
  Using cached toolz-0.12.1-py3-none-any.whl (56 kB)
  Using cached jsonschema-4.21.1-py3-none-any.whl (85 kB)
  Using cached httpcore-1.0.4-py3-none-any.whl (77 kB)
  Using cached rich-13.7.1-py3-none-any.whl (240 kB)
  Using cach